In [32]:
import pandas as pd
import bamboolib
import os
import datetime
import numpy as np
from matplotlib import pyplot
from matplotlib.ticker import MultipleLocator, FixedFormatter, FixedLocator
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sn
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
from IPython.core.display import display, HTML

## Analyse des segments

In [2]:
aymeric =  "/home/aymeric/python-scripts/anses_medialab/datas/" #aymeric
jp = 'd:/Projects/Medialab/Anses/' # Jean Philippe


path_base = aymeric
file_segmentation = f"{path_base}segmentation_common_freq.csv"

index=f"{path_base}index_alldefault_nameexhaustive_aymeric.csv"

all_corpus = f"{path_base}all_corpus_seg_with_sdhi_PI.csv"

#twittos = f"{path_base}tweets_pesticides/500_first_twittos_on_pesticides.csv"

data_file = path_base+'fb_tw_med_data.csv.gz'#,line_terminator='\n',index=False)


In [3]:
list_base = ["index", "all_corpus"]
which_base = list_base[0]

In [4]:
if which_base == "all_corpus":
    df0 = pd.read_csv(all_corpus, sep = "\t")
else:
    df_index = pd.read_csv(index, sep = "\t")
    dfseg = pd.read_csv(file_segmentation, sep = "\t")

    df_index['date'] = pd.to_datetime(df_index['date'], infer_datetime_format=True)
    df_index['yearmonth']=(df_index['date'].dt.strftime('%Y-%m'))
    df_index['date'] = df_index['date'].dt.date
    
    segment = dfseg[["yearmonth", "segm", "origin"]]
    df0 = df_index.merge(segment, how = "inner", on = ["origin", "yearmonth"])
    df0["start_segment"] = df0.groupby(['origin','segm'])["date"].transform('min')
    df0["end_segment"] = df0.groupby(['origin','segm'])["date"].transform('max')

In [6]:
dft = pd.read_csv(data_file)

In [7]:
# Step: Rename column
segment = df0[["yearmonth", "segm", "origin"]]
dft = dft.rename(columns={'id': 'doc_id'})

dft['date'] = pd.to_datetime(dft['date'], infer_datetime_format=True)

dft['yearmonth']=(dft['date'].dt.strftime('%Y-%m'))
dft['date'] = dft['date'].dt.date
    
segment = dfseg[["yearmonth", "segm", "origin"]]
dft0 = dft.merge(segment, how = "inner", on = ["origin", "yearmonth"])
dft0["start_segment"] = dft0.groupby(['origin','segm'])["date"].transform('min')
dft0["end_segment"] = dft0.groupby(['origin','segm'])["date"].transform('max')



## Facebook

In [8]:
dfseg0 = dfseg.copy()

In [9]:
def filter_data(data, domain, segment):
    dfseg = data[0]
    dfseg1 = dfseg.loc[(dfseg["origin"] == domain) & (dfseg["segm"] == segment)]
    
    df = data[1]
    df1 = df.loc[(df["origin"] == domain) & (df["segm"] == segment)]
    
    dft = data[2]
    dft1 = dft.loc[(dft["origin"] == domain) & (dft["segm"] == segment)]
    
    return dfseg1, df1, dft1

In [54]:
domain = ["facebook", "twitter", "media"]
segment = 0
list_df = [dfseg0, df0, dft0]
dfseg1, df1, dft1 = filter_data(list_df, domain = domain[0], segment = 0)

In [56]:
dft1['doc_id'] = dft1['doc_id'].astype('string')

In [57]:
specific_terms = [x.strip() for x in dfseg1["top20_term_chi2_True"].iloc[0].split(",")]

print(specific_terms)

df1.loc[df1["term"].isin(specific_terms) == True, "in_top20_term"] = 1
df1.loc[df1["term"].isin(specific_terms) == False, "in_top20_term"] = 0
df2 = df1.drop_duplicates(subset = ["doc_id", "term"]).groupby(['doc_id', 'source', "date"]).agg(sum_of_top20_term =('in_top20_term', 'sum')).reset_index()

df2 = df2.merge(dft1[["doc_id", "text"]], how = "inner", on = ["doc_id"])

['céréalier charentais', 'monde selon monsanto', 'jeune femme', 'paul françois', 'monsanto', 'lasso', 'procès', 'céréaliers', 'parasite', 'pesticides', 'résidus de pesticides', 'victimes de pesticides', 'effet', 'roundup', 'miel', 'générations futures', 'syngenta', 'victimes', 'agriculture biologique', 'autorité européenne']


In [58]:
segment

0

In [99]:
moyenne = np.mean(df2["sum_of_top20_term"]) 
df3 = df2.loc[df2["sum_of_top20_term"] >= 2 ]
df3 = df3.sort_values(by=['date'], ascending=[True])

In [100]:
moyenne + (moyenne*0.95)

2.767741935483871

In [136]:
nombre_texts = len(df3["text"])

for i, text in enumerate(df3["text"]):
    doc_id = df3["doc_id"].iloc[i]
    list_term = [x for x in df1["term"].loc[(df1["doc_id"] == doc_id)].unique()]
    list_top20_term = [x for x in df1["term"].loc[(df1["doc_id"] == doc_id) & (df1["in_top20_term"] == 1)].unique()]
    list_top20_term_absent = set(specific_terms) - set(list_top20_term)
    date = df3["date"].iloc[i].strftime("%Y-%m-%d")
    auteur = df3["source"].iloc[i]
    display(HTML(f"<h2 style='text-align:center;'> Texte {doc_id} ({i+1} sur {nombre_texts})</h2>"))
    display(HTML(f"<strong>Segment :</strong> {segment}"))
    display(HTML(f"<strong>Date :</strong> {date}"))
    display(HTML(f"<strong>Auteur :</strong> {auteur} \n"))
    print(f"Termes indexés : {list_term} \n")
    print(f"Termes spécifiques présents : {list_top20_term} \n")
    print(f"Termes spécifiques absents : {list_top20_term_absent} \n")
    try:
        for j, term in enumerate(list_term):
            if term not in list_top20_term:
                text = re.sub(term, f'<mark style="background-color:yellow">{term}</mark>', text, flags=re.I)  
            else:
                text = re.sub(term, f'<mark style="background-color:pink">{term}</mark>', text, flags=re.I)  
        display(HTML(f'{text}'))
    except:
        print(f"{text} \n")
   

Termes indexés : ['effet', 'roundup', 'tribunal', 'santé', 'décision', 'herbicide', 'culture', 'ogm', 'maïs'] 

Termes spécifiques présents : ['effet', 'roundup'] 

Termes spécifiques absents : {'monsanto', 'pesticides', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'parasite', 'céréaliers', 'résidus de pesticides', 'procès', 'victimes'} 



Termes indexés : ['santé', 'effet', 'roundup', 'cour', 'décision', 'herbicide', 'pesticides', 'plantes'] 

Termes spécifiques présents : ['effet', 'roundup', 'pesticides'] 

Termes spécifiques absents : {'monsanto', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'parasite', 'céréaliers', 'résidus de pesticides', 'procès', 'victimes'} 



Termes indexés : ['pesticides', 'monsanto'] 

Termes spécifiques présents : ['pesticides', 'monsanto'] 

Termes spécifiques absents : {'roundup', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'parasite', 'céréaliers', 'résidus de pesticides', 'procès', 'victimes', 'effet'} 



Termes indexés : ['écologiste', 'agriculteur', 'alimentation', 'aliments', 'campagne', 'pesticides', 'problème', 'algue', 'bretagne', 'systèmes', 'victimes', 'planète'] 

Termes spécifiques présents : ['pesticides', 'victimes'] 

Termes spécifiques absents : {'roundup', 'monsanto', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'parasite', 'céréaliers', 'résidus de pesticides', 'procès', 'effet'} 



Termes indexés : ['monde', 'europe', 'écologie', 'écologiste', 'campagne', 'pesticides', 'agriculteur', 'problème', 'algue', 'bretagne', 'systèmes', 'victimes', 'planète'] 

Termes spécifiques présents : ['pesticides', 'victimes'] 

Termes spécifiques absents : {'roundup', 'monsanto', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'parasite', 'céréaliers', 'résidus de pesticides', 'procès', 'effet'} 



...

Termes indexés : ['pesticides', 'aliments', 'résidus', 'résidus de pesticides', 'vitamines'] 

Termes spécifiques présents : ['pesticides', 'résidus de pesticides'] 

Termes spécifiques absents : {'roundup', 'monsanto', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'parasite', 'céréaliers', 'procès', 'victimes', 'effet'} 



Termes indexés : ['parasite', 'air', 'pollution', 'population', 'pesticides', 'salariés'] 

Termes spécifiques présents : ['parasite', 'pesticides'] 

Termes spécifiques absents : {'roundup', 'monsanto', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'céréaliers', 'résidus de pesticides', 'procès', 'victimes', 'effet'} 



Termes indexés : ['monsanto', 'pesticides', 'recherche', 'abeilles'] 

Termes spécifiques présents : ['monsanto', 'pesticides'] 

Termes spécifiques absents : {'roundup', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'parasite', 'céréaliers', 'résidus de pesticides', 'procès', 'victimes', 'effet'} 



Termes indexés : ['pesticides', 'effet'] 

Termes spécifiques présents : ['pesticides', 'effet'] 

Termes spécifiques absents : {'roundup', 'monsanto', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'parasite', 'céréaliers', 'résidus de pesticides', 'procès', 'victimes'} 



Termes indexés : ['ogm', 'herbicide', 'roundup', 'roundup de monsanto', 'monsanto', 'betterave', 'culture'] 

Termes spécifiques présents : ['roundup', 'monsanto'] 

Termes spécifiques absents : {'pesticides', 'victimes de pesticides', 'paul françois', 'céréalier charentais', 'agriculture biologique', 'syngenta', 'autorité européenne', 'monde selon monsanto', 'jeune femme', 'lasso', 'miel', 'générations futures', 'parasite', 'céréaliers', 'résidus de pesticides', 'procès', 'victimes', 'effet'} 



In [86]:
text = df2['text'].loc[(df2["doc_id"] == "413916")]
for x in text :
    print(x)

http://www.menustoxiques.fr/pdf/doc_menubio_151211.pdf

menustoxiques.fr
Nouvelle étude comparative de Générations Futures sur les pesticides dans les aliments conventionnels comparés aux mêmes aliments Bio ! http://www.menustoxiques.fr/pdf/doc_menubio_151211.pdf


In [133]:
text = """Santé Canada reçoit l'ordre de réexaminer les effets environnementaux du Roundup | equiterre.org 
- P La Cour fédérale du Canada a ordonné à Santé Canada de réévaluer sa décision de ne pas examiner les effets 
d'un herbicide, commercialisé sous le nom de Roundup. Ce pesticide est un des plus populaires sur le marché et 
il accompagne une proportion importante des plantes modifiées génétiquement utilis... equiterre.org 
Santé Canada reçoit l'ordre de la Cour fédérale de réexaminer les effets environnementaux du RoundUp!"""

In [134]:
list_term = ["pesticides", "roundup"]

In [135]:
import re
for j, term in enumerate(list_term):
    if term[-1]== "s":
        singular_term = term[0:-1]
    else:
        singular_term = term
    #text = text.replace(term[0:len(term)-1], f'<mark style="background-color:yellow">{term}</mark>')
    text = re.sub(term, f'<mark style="background-color:yellow">{term}</mark>', text, flags=re.I)          #

display(HTML(f'{text}'))